In [10]:
from lib.Analysis import Analysis
import numpy as np
analysis = Analysis()

In [11]:
import pandas as pd
all_data = analysis.get_all_json_files()
dataframe = []

for filename in all_data:
  locais = all_data[filename]
  for local in locais:
    dataframe.append({
      'nome': local.nome,
      'preco': local.preco,
      'time': local.time,
      'owner_cresci': local.owner.cresci,
      'owner_nome': local.owner.nome,
      'owner_id': local.owner.cresci,
      'celular': local.owner.celular,
      'telefone': local.owner.telefone,
      'origem': local.origin,
      'latitude': local.lat,
      'longitude': local.lon,
    })

[<entities.Empreendimento.Local object at 0x0000015CE4733950>, <entities.Empreendimento.Local object at 0x0000015CE4733EF0>, <entities.Empreendimento.Local object at 0x0000015CE4747EC0>, <entities.Empreendimento.Local object at 0x0000015CE4747B90>, <entities.Empreendimento.Local object at 0x0000015CE47477D0>, <entities.Empreendimento.Local object at 0x0000015CE4747410>, <entities.Empreendimento.Local object at 0x0000015CE4747050>, <entities.Empreendimento.Local object at 0x0000015CE4746C90>, <entities.Empreendimento.Local object at 0x0000015CE4746960>, <entities.Empreendimento.Local object at 0x0000015CE4746600>, <entities.Empreendimento.Local object at 0x0000015CE47463C0>, <entities.Empreendimento.Local object at 0x0000015CE4746090>, <entities.Empreendimento.Local object at 0x0000015CE4745CD0>, <entities.Empreendimento.Local object at 0x0000015CE4745A60>, <entities.Empreendimento.Local object at 0x0000015CE47456D0>, <entities.Empreendimento.Local object at 0x0000015CE47453A0>, <entiti

In [12]:
df = pd.DataFrame(dataframe)
print(df)

                                                  nome    preco          time  \
0              Apartamento com 2 Quartos à venda, 82m²  1050000  1.699112e+09   
1              Apartamento com 2 Quartos à venda, 60m²   184520  1.699112e+09   
2                     Casa com 3 Quartos à venda, 95m²   485000  1.699112e+09   
3      Casa de Condomínio com 3 Quartos à venda, 280m²  2490000  1.699112e+09   
4                          Lote/Terreno à venda, 360m²   645000  1.699112e+09   
...                                                ...      ...           ...   
15395                Casa com 2 Quartos à venda, 166m²   550000  1.699064e+09   
15396         Apartamento com 2 Quartos à venda, 520m²   212000  1.699064e+09   
15397             Sobrado com 3 Quartos à venda, 161m²   620000  1.699064e+09   
15398                Casa com 2 Quartos à venda, 285m²   260000  1.699064e+09   
15399                     com 5 Quartos à venda, 300m²   630000  1.699064e+09   

      owner_cresci         